In [1]:
%matplotlib widget

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np
import os

In [9]:
def process_data(file):
    coord = []
    for point in file.readlines():
        coord.append(list(map(float,point.split())))
        
    return np.array(coord)

In [39]:
def read_directory(directory):
    point_array_all = []
    land_mark_array_all=[]

    for filename in sorted(os.listdir(directory)):

        if filename.endswith(".pts"): 
            path=os.path.join(directory, filename)
            #print("Processing:",path)
            points = open(path)
            point_array = process_data(points)

            point_array_sampled = []

            for i in range(0,len(point_array),20):
                point_array_sampled.append(point_array[i])

            point_array_sampled = np.array(point_array_sampled)

            point_array_all.append(point_array_sampled)

            continue
        elif filename.endswith(".lan"):
            path = os.path.join(directory, filename)
            #print("Processing:",path)
            landmarks = open(path)
            landmark_array = process_data(landmarks)

            land_mark_array_all.append(landmark_array)

            continue
            
    return point_array_all,land_mark_array_all

In [108]:
def calculate_transformation_matrix(original, aligned):
    transform_matrices=[]
    for i in range(0, len(original)):
        transform_matrix = np.zeros(shape=(3,4))
        
        transform_matrix[0][-1] = aligned[i][0][0] - original[i][0][0]
        transform_matrix[0][0] = 1
        
        transform_matrix[1][-1] = aligned[i][0][1] - original[i][0][1]
        transform_matrix[1][1] = 1
        
        transform_matrix[2][-1] = aligned[i][0][2] - original[i][0][2]
        transform_matrix[2][2] = 1
        
        transform_matrices.append(transform_matrix)
    
    return transform_matrices

In [139]:
def calculate_aligned_landmarks(landmarks, transform_matrices):
    
    updated_landmarks_all = []
    
    for i in range(0, len(landmarks)):
        
        updated_landmarks=[]
        
        if(len(landmarks[i])!=0):
            landmarks_1 = np.c_[landmarks[i],np.ones(len(landmarks[i]))]
                
            for j in range(0, len(landmarks[i])):
                
                updated_landmark=[]
                
                for row in transform_matrices[i]:
                    updated_landmark.append(np.dot(landmarks_1[j],row))
           
                
                updated_landmarks.append(updated_landmark)
            updated_landmarks_all.append(updated_landmarks)
        
    return np.array(updated_landmarks_all, dtype=object)

In [140]:
directory_aligned = "Datasets/Original Foot Data/Bounding Box Aligned/"
directory_processed = "Datasets/Original Foot Data/Processed/"

In [141]:
aligned_points_all = read_directory(directory_aligned)[0]
original_points_all, original_landmarks_all = read_directory(directory_processed)

In [142]:
trans_mats = calculate_transformation_matrix(original_points_all, aligned_points_all)

In [143]:
aligned_landmarks_all = calculate_aligned_landmarks(original_landmarks_all,trans_mats)